In [1]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
from elasticsearch import Elasticsearch

In [3]:
import getpass
es_cloud_id = getpass.getpass("Enter Elastic Cloud ID: ")
api_id = getpass.getpass("Enter Elastic API ID: ")
api_key = getpass.getpass("Enter Elastic API KEY: ")

# Create the client instance
client = Elasticsearch(cloud_id=es_cloud_id, api_key=(api_id, api_key))
client.info()

Enter Elastic Cloud ID: ··········
Enter Elastic API ID: ··········
Enter Elastic API KEY: ··········


ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': '5089524370ab4e1396e3270ca581c541', 'cluster_uuid': 'LFC-6cmGSsqshnnawgItBg', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
es_model_id = 'intfloat__multilingual-e5-base'

In [5]:
def site_search(query):

    response = client.search(
        index="site",
        knn={
            "field": "information_embedding.predicted_value",
            "query_vector_builder": {
                "text_embedding": {
                    "model_id": es_model_id,
                    "model_text": f"query: {query}"
                }
            },
            "k": 5,
            "num_candidates": 20,
        }
    )

    formatted_results = []
    for hit in response["hits"]["hits"]:
        result = {
            "score": hit["_score"],
            "관광지명": hit["_source"]["name"],
            "주소": hit["_source"]["address"],
            "관광지설명": hit["_source"]["information"]
        }
        formatted_results.append(result)

    return formatted_results

In [23]:
from datetime import datetime

def culture_search(query):

    response = client.search(
        index="culture",
        knn={
            "field": "information_embedding.predicted_value",
            "query_vector_builder": {
                "text_embedding": {
                    "model_id": es_model_id,
                    "model_text": f"query: {query}"
                }
            },
            "k": 5,
            "num_candidates": 20,
        },
        query={
            "bool": {
                "filter": [
                    {
                        "range": {
                            "startDate": {
                                "lte": datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
                            }
                        }
                    },
                    {
                        "range": {
                            "endDate": {
                                "gte": datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
                            }
                        }
                    }
                ]
            }
        }
    )

    formatted_results = []
    for hit in response["hits"]["hits"]:
        result = {
            "score": hit["_score"],
            "이름": hit["_source"]["name"],
            "지역": hit["_source"]["area"],
            "주소": hit["_source"]["address"],
            "문화정보": hit["_source"]["information"],
            "가격정보": hit["_source"]["price"],
            "시작일": hit["_source"]["startDate"],
            "종료일": hit["_source"]["endDate"],
            "위치정보 : 위도": hit["_source"]['location']["lat"],
            "위치정보 : 경도": hit["_source"]["location"]["lon"]
        }
        formatted_results.append(result)

    return formatted_results

In [27]:
culture_search("서울 미술관 전시")

[{'score': 0.9403391,
  '이름': 'MMCA 서울 상설전《한국현대미술 하이라이트》',
  '지역': '서울',
  '주소': '서울특별시 종로구 삼청로 30 국립현대미술관 서울관',
  '문화정보': '[전시] 서울 종로구 : 국립현대미술관 서울관 : MMCA 서울 상설전《한국현대미술 하이라이트》',
  '가격정보': '2,000원',
  '시작일': '2025-05-01T00:00:00.000000Z',
  '종료일': '2026-05-03T00:00:00.000000Z',
  '위치정보 : 위도': 37.5786274905286,
  '위치정보 : 경도': 126.98010361777376},
 {'score': 0.9387541,
  '이름': '마음_봄',
  '지역': '서울',
  '주소': '서울특별시 종로구 삼청로 30 국립현대미술관 서울관',
  '문화정보': '[전시] 서울 종로구 : 국립현대미술관 서울관 : 마음_봄',
  '가격정보': '무료',
  '시작일': '2025-05-02T00:00:00.000000Z',
  '종료일': '2026-02-27T00:00:00.000000Z',
  '위치정보 : 위도': 37.5786274905286,
  '위치정보 : 경도': 126.98010361777376},
 {'score': 0.9385853,
  '이름': '세르주 블로크展',
  '지역': '서울',
  '주소': '서울특별시 서초구 남부순환로 2406 예술의전당(한가람미술관)',
  '문화정보': '[전시] 서울 서초구 : 예술의전당(한가람미술관) : 세르주 블로크展',
  '가격정보': '일반 (만19~64세) 15,000원 | 청소년, 어린이, 유아 10,000원 (만 24개월~18세) * 유아~초등학생 입장 시 보호자 최소 1명 동반 필수',
  '시작일': '2025-05-29T00:00:00.000000Z',
  '종료일': '2025-08-17T00:00:00.000000Z',
  '위치정보 : 위도'

In [28]:
site_search("전북 자연 관광")

[{'score': 0.9348736,
  '관광지명': '완주전통문화체험장',
  '주소': '',
  '관광지설명': '전북특별자치도 완주군 : 완주전통문화체험장 : 고즈넉한 한옥의 숨결을 느끼며 뛰어난 자연경관 속에 아늑한 휴식을 즐기기에 안성맞춤인 공간'},
 {'score': 0.934268,
  '관광지명': '금강호관광지',
  '주소': '전북특별자치도 군산시 성산면 성덕리 411-1',
  '관광지설명': '전북특별자치도 군산시 : 금강호관광지 : 국내 최대 철새도래지로서 자연과 문화가 어우러지는 사계절 관광지로 거듭나고 있음'},
 {'score': 0.93109846,
  '관광지명': '고산자연휴양림',
  '주소': '',
  '관광지설명': '전북특별자치도 완주군 : 고산자연휴양림 : 숲에서 산림욕을 즐길 수 있는 사계절 가족휴양지'},
 {'score': 0.93071413,
  '관광지명': '변산해수욕장',
  '주소': '전북특별자치도 부안군 변산면 대항리 523-117',
  '관광지설명': '전북특별자치도 부안군 : 변산해수욕장 : 국립공원 변산반도에 위치한 노을이 아름다운 해수욕장'},
 {'score': 0.92856264,
  '관광지명': '웅포관광지',
  '주소': '전북특별자치도 익산시 웅포면 웅포리 727',
  '관광지설명': '전북특별자치도 익산시 : 웅포관광지 : 웅포 곰개나루 및 캠핑장'}]